In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy import stats

In [ ]:
data_path = ''
room_type = 'living_room'
method = 'tsne'
mode = 'test'
PATH = f'data/external_dataset_comparison/cities_large_test_int_ext_{room_type}_{method}.csv'

In [ ]:
#specify index of join point for external vs airbnb data
join_point = 269

In [43]:
dataframe = pd.read_csv(PATH)
dataframe_ext = dataframe.iloc[:join_point]
dataframe_airbnb = dataframe.iloc[join_point:]

In [44]:
dataframe_ext = dataframe.iloc[:join_point]
dataframe_airbnb = dataframe.iloc[join_point:]

In [45]:
dataframe['label'].unique()

array(['blocket_sweden', 'zigbang_seoul', 'craigslist_newYork',
       'craigslist_losAngeles', 'ohou_seoul', 'craigslist_chicago',
       'zillow_chicago', 'mexicoCity', 'newYork', 'losAngeles', 'toronto',
       'chicago', 'saoPaulo', 'buenosAires', 'rio', 'lima', 'bogota',
       'london', 'berlin', 'madrid', 'milan', 'paris', 'kinshasa',
       'cairo', 'abidjan', 'nairobi', 'capeTown', 'tokyo', 'seoul',
       'osaka', 'hongKong', 'bangkok', 'delhi', 'mumbai', 'dhaka',
       'chennai', 'istanbul', 'riyadh', 'ankara', 'dubai', 'lahore',
       'sydney', 'melbourne', 'auckland', 'jakarta', 'adelaide',
       'kolkata'], dtype=object)

In [46]:
city_dict = {'seoul':['ohou_seoul','zigbang_seoul'],
            'chicago':['zillow_chicago','craigslist_chicago'],
            'newYork':['craigslist_newYork'],
            'losAngeles':['craigslist_losAngeles']}

In [47]:
dataframe_ext.tail()

,Unnamed: 0,path,label,x,y
264,264,/media/data_16T/nwl/datasets/zillow/chicago/20...,zillow_chicago,10.966833,-35.023064
265,265,/media/data_16T/nwl/datasets/zillow/chicago/20...,zillow_chicago,10.360454,-50.006405
266,266,/media/data_16T/nwl/datasets/zillow/chicago/20...,zillow_chicago,-16.796452,58.213383
267,267,/media/data_16T/nwl/datasets/zillow/chicago/20...,zillow_chicago,47.732080,-13.262972
268,268,/media/data_16T/nwl/datasets/zillow/chicago/20...,zillow_chicago,-12.237353,35.660780


In [48]:
dataframe_airbnb.head()

,Unnamed: 0,path,label,x,y
269,269,/media/data_16T/nwl/datasets/airbnb/mexicoCity...,mexicoCity,-25.757780,19.214460
270,270,/media/data_16T/nwl/datasets/airbnb/mexicoCity...,mexicoCity,7.248037,25.548246
271,271,/media/data_16T/nwl/datasets/airbnb/mexicoCity...,mexicoCity,-54.645440,26.779041
272,272,/media/data_16T/nwl/datasets/airbnb/mexicoCity...,mexicoCity,3.457366,21.361086
273,273,/media/data_16T/nwl/datasets/airbnb/mexicoCity...,mexicoCity,-35.440784,52.970783


In [49]:
def distance_2d(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [ ]:
#compute metrics for checking representativeness of airbnb data
result_data = {'City':[],
               'Room Type':[],
               'External Dataset':[],
               'Samples (Airbnb)':[],
               'Samples (External)':[],
               'Avg Dist to Centroid (Airbnb)':[],
               'Avg Dist to Centroid (External)':[],
               'Difference':[],
               'STD (Airbnb)':[],
               'STD (External)':[],
               'p_val':[]
               }
print("Room_Type:",room_type)
for airbnb_city in city_dict:
    print(airbnb_city)
    print('----------')
    for ext_city in city_dict[airbnb_city]:
        print('------')
        dataframe_airbnb_city = dataframe_airbnb[dataframe_airbnb['label']==airbnb_city]
        dataframe_ext_city = dataframe_ext[dataframe_ext['label']==ext_city]
        
        centroid = (dataframe_airbnb_city['x'].mean(),dataframe_airbnb_city['y'].mean())
        
        distances_ext = []
        distances_airbnb = []
        for i in range(len(dataframe_ext)):
            point = (dataframe_ext.iloc[i]['x'],dataframe_ext.iloc[i]['y'])
            dist = distance_2d(point, centroid)
            distances_ext.append(dist)
        for i in range(len(dataframe_airbnb)):
            point = (dataframe_airbnb.iloc[i]['x'],dataframe_airbnb.iloc[i]['y'])
            dist = distance_2d(point, centroid)
            distances_airbnb.append(dist)
        
        t_stat, p_value_ttest = stats.ttest_ind(distances_ext, distances_airbnb)
        
        print("City:",airbnb_city,"|",len(dataframe_airbnb_city),"samples")
        print("External City",ext_city,"|",len(dataframe_ext_city),"samples")              
        print("Avg dist from airbnb centroid for external images:",np.mean(distances_ext), "STD:",np.std(distances_ext))
        print("Avg dist from airbnb centroid for airbnb images:",np.mean(distances_airbnb), "STD:",np.std(distances_airbnb))
        print("Difference in average distance:",(np.mean(distances_ext))-(np.mean(distances_airbnb)))
        print("P_Value (t_test)",p_value_ttest)

        result_data['City'].append(airbnb_city)
        result_data['Room Type'].append(room_type)
        result_data['External Dataset'].append(ext_city)
        result_data['Samples (Airbnb)'].append(len(dataframe_airbnb_city))
        result_data['Samples (External)'].append(len(dataframe_ext_city))
        result_data['Avg Dist to Centroid (Airbnb)'].append(np.mean(distances_airbnb))
        result_data['Avg Dist to Centroid (External)'].append(np.mean(distances_ext))
        result_data['Difference'].append((np.mean(distances_ext))-(np.mean(distances_airbnb)))
        result_data['STD (Airbnb)'].append(np.std(distances_airbnb))
        result_data['STD (External)'].append(np.std(distances_ext))
        result_data['p_val'].append(p_value_ttest)
        
        print('------')
        
    print('----------')

Room_Type: living_room
seoul
----------
------
City: seoul | 400 samples
External City ohou_seoul | 111 samples
Avg dist from airbnb centroid for external images: 45.6608596523002 STD: 22.78063654088473
Avg dist from airbnb centroid for airbnb images: 51.854325807617535 STD: 21.016363905516688
Difference in average distance: -6.193466155317331
P_Value (t_test) 1.7847751793084721e-06
------
------
City: seoul | 400 samples
External City zigbang_seoul | 9 samples
Avg dist from airbnb centroid for external images: 45.6608596523002 STD: 22.78063654088473
Avg dist from airbnb centroid for airbnb images: 51.854325807617535 STD: 21.016363905516688
Difference in average distance: -6.193466155317331
P_Value (t_test) 1.7847751793084721e-06
------
----------
chicago
----------
------
City: chicago | 400 samples
External City zillow_chicago | 45 samples
Avg dist from airbnb centroid for external images: 51.62172601437098 STD: 19.417992424437738
Avg dist from airbnb centroid for airbnb images: 56.1

In [51]:
metrics_df = pd.DataFrame(result_data)
metrics_df.head()

,City,Room Type,External Dataset,Samples (Airbnb),Samples (External),Avg Dist to Centroid (Airbnb),Avg Dist to Centroid (External),Difference,STD (Airbnb),STD (External),p_val
0,seoul,living_room,ohou_seoul,400,111,51.854326,45.660860,-6.193466,21.016364,22.780637,0.000002
1,seoul,living_room,zigbang_seoul,400,9,51.854326,45.660860,-6.193466,21.016364,22.780637,0.000002
2,chicago,living_room,zillow_chicago,400,45,56.107740,51.621726,-4.486014,25.199060,19.417992,0.003705
3,chicago,living_room,craigslist_chicago,400,38,56.107740,51.621726,-4.486014,25.199060,19.417992,0.003705
4,newYork,living_room,craigslist_newYork,400,8,54.810681,50.488766,-4.321915,24.135590,18.243287,0.003501


In [52]:
metrics_df.to_csv(f'data/metrics_result/{room_type}_{method}.csv')